In [2]:
import os
import openai

# 환경변수에서 OpenAI API 키를 읽어옴.
openai_api_key = os.getenv('OPENAI_API_KEY')

# API 키가 환경변수에 설정되어 있는지 확인.
if not openai_api_key:
    raise ValueError("환경변수 'OPENAI_API_KEY'가 설정되지 않았습니다.")

# OpenAI API 키를 설정.
openai.api_key = openai_api_key

# OpenAI 클라이언트를 생성.
client = openai.OpenAI()

speech_output_path="./roller_voice.mp3"

audio_file = open(speech_output_path, "rb")
transcript=client.audio.transcriptions.create(
    model="whisper-1",
    file=audio_file
)
print(transcript.text)

인생은 롤러코스터와 같아. 둘 다 오르막과 내리막이 있으니까. 하지만 두려움에 떨거나 즐기는 것은 너의 선택이야.


In [9]:
import torch
from transformers import pipeline, WhisperProcessor, WhisperForConditionalGeneration

device="cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32
model_id = "openai/whisper-large-v2"
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-large-v2")
model.config.forced_decoder_ids = None
model.to(device)
processor = WhisperProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    chunk_length_s=30,
    batch_size=16,
    return_timestamps=True,
    torch_dtype=torch_dtype,
    device=device,
)

Device set to use cpu


In [10]:
# 파일 변환
speech_output_path="./roller_voice.mp3"
result_openai = pipe(speech_output_path)
print(result_openai["text"])

C:\Users\niceq\anaconda3\envs\ai_jun\lib\site-packages\transformers\models\whisper\generation_whisper.py:512: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass

 인생은 롤러코스터와 같아. 둘 다 오르막과 내리막이 있으니까. 하지만 두려움에 떨거나 즐기는 것은 너의 선택이야.


In [11]:
result_trans = pipe(speech_output_path, generate_kwargs={"task":"translate"})
print(result_trans)

You have passed task=translate, but also have set `forced_decoder_ids` to [[1, None], [2, 50359]] which creates a conflict. `forced_decoder_ids` will be ignored in favor of task=translate.


{'text': " Life is like a roller coaster. Both have ups and downs. But it's your choice to tremble or enjoy the fear.", 'chunks': [{'timestamp': (0.0, 12.68), 'text': " Life is like a roller coaster. Both have ups and downs. But it's your choice to tremble or enjoy the fear."}]}


In [14]:
from gtts import gTTS

message = "안녕하세요, 오늘부터 영하 10도의 추운 겨울이 시작된대요. 감기 조심하세요."
tts = gTTS(message, lang="ko")
speech_output_path="./gtts_output.mp3"
tts.save(speech_output_path)

In [15]:
from IPython.display import Audio, display
audio_widget = Audio(speech_output_path)
display(audio_widget)

In [19]:
import os
import openai

# 환경변수에서 OpenAI API 키를 읽어옴.
openai_api_key = os.getenv('OPENAI_API_KEY')

# API 키가 환경변수에 설정되어 있는지 확인.
if not openai_api_key:
    raise ValueError("환경변수 'OPENAI_API_KEY'가 설정되지 않았습니다.")

# OpenAI API 키를 설정.
openai.api_key = openai_api_key

# OpenAI 클라이언트를 생성.
client = openai.OpenAI()

message = "추운 겨울 감기 조심하세요! 메리 크리스마스~"

response = client.audio.speech.create(
    model="tts-1",
    voice="fable",
    input=message,
)
speech_out_path="./openai_tts_output.mp3"
with open(speech_out_path, "wb") as audio_file:
    audio_file.write(response.content)

In [20]:
from IPython.display import Audio, display
audio_widget = Audio(speech_output_path)
display(audio_widget)

In [2]:
import os
import openai

# 환경변수에서 OpenAI API 키를 읽어옴.
openai_api_key = os.getenv('OPENAI_API_KEY')

# API 키가 환경변수에 설정되어 있는지 확인.
if not openai_api_key:
    raise ValueError("환경변수 'OPENAI_API_KEY'가 설정되지 않았습니다.")

# OpenAI API 키를 설정.
openai.api_key = openai_api_key

# OpenAI 클라이언트를 생성.
client = openai.OpenAI()

from langchain.document_loaders import PyPDFLoader
def pdf_loader(pdf_path):
    loader=PyPDFLoader(pdf_path)
    pdf_doc=loader.load()
    return pdf_doc

pdf_path = "C:/Users/niceq/Desktop/차은우_비주얼_나무위키.pdf"
pages_content = pdf_loader(pdf_path)[0].page_content # 첫 번째 페이지

system_ai_en = """ You are an artificial intelligence assistance, and you should be able to respond to various topics such as reading PDFs,
summarizing documents, managing schedules, providing weather information, searching for the shortest route, and conducting web searches.
Please respond briefly in Korean."""

prompt = """다음은 문서 중 일부 내용입니다.
{pages_content}
이 문서의 주요 내용을 요약해 주세요.
답변:"""

completion = client.chat.completions.create(
    model="gpt-3.5-turbo-1106",
    messages=[
        {"role": "system", "content": system_ai_en},
        {"role": "user", "content": prompt.format(pages_content=pages_content)},
    ]
)
print(completion.choices[0].message.content)

최근 인기를 끌고 있는 아이돌 차은우의 외모가 여러 팬덤 사이에서 대표적으로 언급되고 칭찬받고 있으며, 그의 비주얼은 남성들 사이에서도 많은 찬양을 받고 있다. 이로 인해 ASTRO의 인지도가 상승하였다는 내용입니다.


In [3]:
import os
import openai

# 환경변수에서 OpenAI API 키를 읽어옴.
openai_api_key = os.getenv('OPENAI_API_KEY')

# API 키가 환경변수에 설정되어 있는지 확인.
if not openai_api_key:
    raise ValueError("환경변수 'OPENAI_API_KEY'가 설정되지 않았습니다.")

# OpenAI API 키를 설정.
openai.api_key = openai_api_key

# OpenAI 클라이언트를 생성.
client = openai.OpenAI()

from langchain.document_loaders import PyPDFLoader
def pdf_loader(pdf_path):
    loader=PyPDFLoader(pdf_path)
    pdf_doc=loader.load()
    return pdf_doc

pdf_path = "C:/Users/niceq/Desktop/차은우_비주얼_나무위키.pdf"
pages_content = pdf_loader(pdf_path)
print(pages_content[0].page_content[:200])

잘생김 대명사 
 
 
- 아이돌/스포츠 팬덤에서는 최애를 '시대를 대표하는 미남'의 이름으로 부르는 
주접 문화가 있는데 최근에는 차은우가 가장 많이 언급된다. 
( ex. 유튜브 아이콘 유튜브 아이콘 유튜브 아이콘 유튜브 아이콘 인스타그램 
아이콘 X Corp 아이콘(블랙) X Corp 아이콘(블랙) X Corp 아이콘(블랙) X Corp 
아이콘(블랙


In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
# 스플리터 설정
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=2000,
    chunk_overlap=0,
)
# pages_content 내용 분할
split_docs = text_splitter.split_documents(pages_content)
# 분할된 문서의 수 확인
print("분할된 문서의 개수는: ", len(split_docs))

분할된 문서의 개수는:  4


In [14]:
## 각각의 문서에 대한 요약 실행
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain

# map template 설정, {pages_content} 분할된 내용이 입력
map_template = """ 다음은 문서 중 일부 내용입니다.
{pages_content}
이 문서의 주요 내용을 입력해 주세요.
"""
# map 기본 프롬프트
map_prompt = PromptTemplate.from_template(map_template)
# 문서 내용이 길 수 있으므로 model을 gpt-3.5-turbo-0125 설정
llm = ChatOpenAI(temperature=0,
                 model_name='gpt-3.5-turbo-0125')
map_chain = LLMChain(llm=llm, prompt=map_prompt)

# reduce 단계에서 처리할 프롬프트 정의
reduce_template = """ 다음은 문서 요약의 집합입니다.
{summaries}
이 내용을 바탕으로 통합된 문서 요약을 작성해 주세요.
"""
# Reduce 프롬프트
reduce_prompt = PromptTemplate.from_template(reduce_template)

# Reduce에서 수행할 LLMChain 정의
reduce_chain = LLMChain(llm=llm, prompt=reduce_prompt)

In [11]:
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains import ReduceDocumentsChain

# 문서 목록 통합 체인 설정
combine_doc_chain = StuffDocumentsChain(
    llm_chain=reduce_chain,
    document_variable_name="summaries" # reduce 프롬프트에 대입되는 변수
)

# 분할된 문서를 순차적으로 reduce 처리
reduce_doc_chain = ReduceDocumentsChain(
    combine_documents_chain=combine_doc_chain,
    collapse_documents_chain=combine_doc_chain,
    token_max=4000,
)

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_2492\91324474.py:11: LangChainDeprecationWarning: This class is deprecated. Please see the migration guide here for a recommended replacement: https://python.langchain.com/docs/versions/migrating_chains/map_reduce_chain/
  reduce_doc_chain = ReduceDocumentsChain(


In [16]:
from langchain.chains import MapReduceDocumentsChain

# 최종 체인 연결
final_chain=MapReduceDocumentsChain(
    llm_chain=map_chain, # 각 문서 맵핑
    reduce_documents_chain=reduce_doc_chain,
    document_variable_name="pages_content",
    return_intermediate_steps=False,
)
# 최종 결과 실행
result_summary=final_chain.run(split_docs)
# 요약 결과 출력
print(result_summary)

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_2492\2886328086.py:11: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result_summary=final_chain.run(split_docs)


차은우는 아이돌/스포츠 팬덤에서 '시대를 대표하는 미남'으로 인기를 끌며, 최근에는 남초 커뮤니티에서도 높은 인기를 얻고 있다. 그의 외모와 매력적인 얼굴은 여성들 사이에서 인기를 끌며, 데뷔 초부터 얼굴천재로 불리기도 했다. 또한, 차은우는 자신의 눈을 가장 마음에 들어하는데, 속눈썹의 영향으로 동화 속 왕자님과 같은 느낌을 준다고 평가받고 있다. 그의 퍼스널 컬러는 여름 쿨톤이라는 추측이 있지만, 어떤 톤이든 다 잘 어울린다고 설명되어 있다.
